In [46]:
import pandas as pd
import numpy as np
import mysql.connector

### Connect to DB and map Tables into DataFrames

In [47]:
class dataframes:
    user=test=questions=options=responses=olap_test=olap_question=pd.DataFrame()
    def __init__(self):
        cnx = mysql.connector.connect(user='root', password='my123sql',
                              host='localhost', database='intest')
        
        query_user = "SELECT * FROM users"
        self.user = pd.read_sql(query_user, cnx)
        
        query_test = "SELECT * FROM test_db"
        self.test= pd.read_sql(query_test, cnx)
        
        query_questions = "SELECT * FROM questions"
        self.questions= pd.read_sql(query_questions, cnx)
        
        query_options = "SELECT * FROM mcq_options"
        self.options= pd.read_sql(query_options, cnx)
        
        query_responses = "SELECT * FROM responses"
        self.responses= pd.read_sql(query_responses, cnx)
        
        query_olap_test = "SELECT * FROM olap_test"
        self.olap_test= pd.read_sql(query_olap_test, cnx)
        
        query_olap_question = "SELECT * FROM olap_question"
        self.olap_question= pd.read_sql(query_olap_question, cnx)
        
        cnx.close()

In [48]:
df=dataframes()

In [49]:
responses=df.responses
idxList=responses.query('testId==1 & userId==3')
idxList.index.tolist()

[3, 4, 5]

In [50]:
df.responses.loc[0].to_dict()

{'id': 1,
 'questionId': 1,
 'userId': 2,
 'testId': 1,
 'body': 'New Delhi',
 'obtainedMarks': 5}

### Run OLAP processes for TESTwise Analysis

In [51]:
class OLAP_Test:
    df:dataframes
    results={'id': 0, 'testId':0, 'totalAppeared':0, 'maxMarks':0, 'highestMarks':0, 'lowestMarks':0,
             'avgMarks':0, 'noOfPassed':0, 'noOfFailed':0, 'lastUpdated': ''}
    testId=0
    
    def __init__(self, df: dataframes, testId: int) -> None:
        self.df=df
        self.testId=testId
        self.results['testId']=testId
        
        
    def calculate(self):
        
        responses=self.df.responses.query('testId==@self.testId')
        test=self.df.test.query('id==@self.testId')
        
        def setTotalAppeared():
            self.results['totalAppeared']=len(responses['userId'].unique())
        
        def setMaxMarks():
            mm=test['maxMarks'].loc[0]
            self.results['maxMarks']=mm
        
        def setHighestMarks():
            max_score = responses.groupby('userId')['obtainedMarks'].sum().max()
            self.results['highestMarks']=max_score
            
        def setLowestMarks():
            min_score = responses.groupby('userId')['obtainedMarks'].sum().min()
            self.results['lowestMarks']=min_score
        
        def setAvgMarks():
            average_marks = responses.groupby('userId')['obtainedMarks'].sum().mean()
            self.results['avgMarks']=average_marks
        
        def setNoOfPassedandFailed():
            passMarks=test.passMarks.loc[0]
            passed=(responses.groupby('userId')['obtainedMarks'].sum() >= passMarks).sum()
            self.results['noOfPassed']=passed
            self.results['noOfFailed']=self.results['totalAppeared']-passed
        setTotalAppeared()
        setMaxMarks()
        setHighestMarks()
        setLowestMarks()
        setAvgMarks()
        setNoOfPassedandFailed()
        return self.results
 

In [52]:
o=OLAP_Test(df=df,testId=1)
result=o.calculate()
print(result)

{'id': 0, 'testId': 1, 'totalAppeared': 4, 'maxMarks': 15, 'highestMarks': 15, 'lowestMarks': 0, 'avgMarks': 7.5, 'noOfPassed': 2, 'noOfFailed': 2, 'lastUpdated': ''}


In [53]:
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

In [54]:
import json
json.dumps(result, default=np_encoder)

'{"id": 0, "testId": 1, "totalAppeared": 4, "maxMarks": 15, "highestMarks": 15, "lowestMarks": 0, "avgMarks": 7.5, "noOfPassed": 2, "noOfFailed": 2, "lastUpdated": ""}'

In [55]:
id=0
testId=1
searchTest=df.olap_test.query('testId==@testId')
if len(searchTest)>0:
    id=searchTest.index()+1
else:
    id=len(df.olap_test)+1
id

1

## OLAP Questions

In [56]:
#analyzes one question by its ID
class OLAP_questions:
    df:dataframes
    result={'id': 0, 'questionId':0, 'maxMarks':0, 'highestMarks':0, 'lowestMarks':0,
             'avgMarks':0, 'topic': ''}
    testId:int
    
    #maps dataframes created and locks the testId
    def __init__(self, df:dataframes, testId: int):
        self.df=df
        self.testId=testId
        
    #calculates analysis for a particular question by questionId
    def calculate(self,questionId:int):
        responses=self.df.responses.query('questionId==@questionId')
        question=self.df.questions.query('id==@questionId')
        self.result['questionId']=questionId
        self.result['topic']=question['topic'].iloc[0]
        self.result['maxMarks']=question['maxMarks'].iloc[0]
        self.result['highestMarks']=responses['obtainedMarks'].max()
        self.result['lowestMarks']=responses['obtainedMarks'].min()
        self.result['avgMarks']=responses['obtainedMarks'].mean()

        return self.result       

In [57]:
questionId=2
responses=df.responses.query('questionId==@questionId')
responses

,id,questionId,userId,testId,body,obtainedMarks
1,2,2,2,1,Narendra Modi,5
4,5,2,3,1,Rahul Gandhi,0
7,14,2,4,1,Narendra Modi,5
10,17,2,5,1,Rahul Gandhi,0


In [58]:
questionId=2
question=df.questions.query('id==@questionId')
topic=question['topic'].iloc[0]
display(question)
topic

,id,optionsId,testId,maxMarks,body,topic,answerType
1,2,2,1,5,Who is the current Prime Minister of India?,Government,mcq


'Government'

In [59]:
olapq=OLAP_questions(df,1)
olapq.calculate(questionId=1)

{'id': 0,
 'questionId': 1,
 'maxMarks': 5,
 'highestMarks': 5,
 'lowestMarks': 0,
 'avgMarks': 2.5,
 'topic': 'Cities'}

## OLAP Student

In [83]:

#analyzes test scores wrt topics



class studentAnalysis:
    testId:int
    df:dataframes
    def __init__(self,df:dataframes) -> None:
        self.df=df
        
    def getTestData(self, testId:int, studentId: int):
        olapTest=OLAP_Test(df=self.df,testId=testId)
        results=olapTest.calculate()
        responses=self.df.responses.query('testId==@testId & userId==@studentId')
        results['studentId']=studentId
        results['totalMarksObtained']=responses['obtainedMarks'].sum()
        return results
    
    def getQuestionData(self, testId:int, studentId: int, questionId: int):
        olapq=OLAP_questions(df=self.df,testId=testId)
        result=olapq.calculate(questionId=questionId)
        response=self.df.responses.query('testId==@testId & userId==@studentId & questionId==@questionId')
        result['obtainedMarks']=response['obtainedMarks'].iloc[0]
        return result

In [85]:
df=dataframes()
sa=studentAnalysis(df)
sa.getTestData(1,2)

{'id': 0,
 'testId': 1,
 'totalAppeared': 4,
 'maxMarks': 15,
 'highestMarks': 15,
 'lowestMarks': 0,
 'avgMarks': 7.5,
 'noOfPassed': 2,
 'noOfFailed': 2,
 'lastUpdated': '',
 'studentId': 2,
 'totalMarksObtained': 15}

In [86]:
testId=1
userId=2
questionId=1
response=df.responses.query('testId==@testId & userId==@userId & questionId==@questionId')
response['obtainedMarks'].iloc[0]

5

In [89]:
q=sa.getQuestionData(testId=1,studentId=2,questionId=1)
print(q)

{'id': 0, 'questionId': 1, 'maxMarks': 5, 'highestMarks': 5, 'lowestMarks': 0, 'avgMarks': 2.5, 'topic': 'Cities', 'obtainedMarks': 5}
